In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.data.make_dataset import load_dossies_df
from src.data.make_metadados import load_dossies_metadados_df
from src.features.build_features import get_tfidf_features, get_tf_features
from src.models import train_model, predict_model, validate_model
from src.visualization.visualize import plot_dendrogram_Agglomerative, plot_dendrogram

In [ ]:
dossies = load_dossies_df()
len(dossies)

In [ ]:
dossies.head()

In [ ]:
dossies_metadados = load_dossies_metadados_df(dossies.file)

In [ ]:
dossies_metadados.head()

In [ ]:
X, tfidf_feature_names = get_tfidf_features(dossies.text)
X.shape

In [ ]:
X_topics, tf_feature_names = get_tf_features(dossies.text, n_features=1000)
X_topics.shape

In [ ]:
aggl_clustering = train_model.agglomerative(X, distance_threshold=0, n_clusters=None, linkage='ward')
aggl_clustering

In [ ]:
n_topics = 100
validate_model.print_top_words(train_model.lda(X_topics, n_topics), tf_feature_names, n_top_words=10)
X_topics

In [ ]:
validate_model.print_top_words(train_model.nmf(X, n_topics), tfidf_feature_names, n_top_words=10)

In [ ]:
plot_dendrogram_Agglomerative(
    aggl_clustering,
    truncate_mode='lastp',  # show only the last p merged clusters
    p=100,  # show only the last p merged clusters
    leaf_rotation=90,
    # leaf_font_size=12.,
    show_contracted=True    
)

In [ ]:
Z = train_model.linkage_matrix(X, method='ward')
plot_dendrogram(
    Z,
    truncate_mode='level',
    p=5,
    leaf_rotation=90,
    # leaf_font_size=12.,
    show_contracted=True
)

In [ ]:
# a partir da analise do dendrograma plotada, 5 clusters parece uma boa escolha
n_clusters = 5
labels = predict_model.Z_labels(Z, t=n_clusters, criterion='maxclust')
labels

In [ ]:
# Com os resultados dessa célula pode-se presumir que documentos com a label 2 está fortemente relacionado à questões de agricultura
cluster_2 = validate_model.checar_categoria(dossies_metadados, labels, 2)
cluster_2.info()
print(cluster_2.head())

In [ ]:
validate_model.checar_substring(cluster_2, 'titulo', 'cultivo')

In [ ]:
# Com os resultados dessa célula pode-se presumir que documentos com a label 1 está fortemente relacionado à questões de agricultura
cluster_1 = validate_model.checar_categoria(dossies_metadados, labels, 1)
cluster_1.head()

In [ ]:
validate_model.checar_substring(cluster_1, 'titulo', r'cogumelos?')